In [1]:
import opticalflow3D
import warnings
from numba.core.errors import NumbaPerformanceWarning

warnings.filterwarnings("ignore", category=NumbaPerformanceWarning)

# Farneback two frame motion estimation

In [2]:
farneback = opticalflow3D.Farneback3D(iters=5,
                                      num_levels=5,
                                      scale=0.5,
                                      spatial_size=7,
                                      presmoothing=7,
                                      filter_type="box",
                                      filter_size=21,
                                     )

In [3]:
# Load the images
image1 = opticalflow3D.helpers.load_image(r"E:\data\3DTFM\test_set\image1.tif")
image2 = opticalflow3D.helpers.load_image(r"E:\data\3DTFM\test_set\image2.tif")

In [4]:
# Realign the image manually to remove any drift during the acquisition of the image
image1 = opticalflow3D.helpers.crop_image(image1, z_reverse=True, z_start=2, y_start=1, x_start=1)
image2 = opticalflow3D.helpers.crop_image(image2, z_reverse=True, z_end=-2, y_end=-1, x_end=-1)

In [5]:
output_vz, output_vy, output_vx, output_confidence = farneback.calculate_flow(image1, image2, 
                                                                              start_point=(0, 300, 300),
                                                                              total_vol=(512, 512, 512),
                                                                              sub_volume=(350, 350, 350),
                                                                              overlap=(64, 64, 64),
                                                                              threadsperblock=(8, 8, 8),
                                                                             )

Running 3D Farneback optical flow with the following parameters:
Iters: 5 | Levels: 5 | Scale: 0.5 | Kernel: 7 | Filter: box-21 | Presmoothing: 7


Item: 4/4: 100%|█████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.06s/it]


In [6]:
opticalflow3D.helpers.save_displacements("vectors.npz", output_vz, output_vy, output_vx)
opticalflow3D.helpers.save_confidence("confidence.npz", output_confidence)

## Perform qualitative/quatitative assessment of vectors

In [7]:
import numpy as np
import skimage.io

In [8]:
start_point=(0, 300, 300)
total_vol=(512, 512, 512)
cropped_image1 = image1[start_point[0]:start_point[0]+total_vol[0],
                        start_point[1]:start_point[1]+total_vol[1],
                        start_point[2]:start_point[2]+total_vol[2]]

cropped_image2 = image2[start_point[0]:start_point[0]+total_vol[0],
                        start_point[1]:start_point[1]+total_vol[1],
                        start_point[2]:start_point[2]+total_vol[2]]    

In [9]:
inverse_image = opticalflow3D.helpers.generate_inverse_image(cropped_image1, output_vx, output_vy, output_vz)

In [11]:
output_image = np.moveaxis(np.stack((inverse_image, cropped_image2)), 0, 1)
skimage.io.imsave(f"comparison_image.tif", output_image, imagej=True)

C:\Users\Xianbin\AppData\Local\Temp\ipykernel_21956\1448891147.py:2: UserWarning: comparison_image.tif is a low contrast image
  skimage.io.imsave(f"comparison_image.tif", output_image, imagej=True)


In [12]:
nrmse = skimage.metrics.normalized_root_mse(cropped_image2, inverse_image)
print(f"Normalized root mse: {nrmse}")

Normalized root mse: 0.0012819914253917673


In [11]:
output_image = np.moveaxis(np.stack((inverse_image, cropped_image2)), 0, 1)
skimage.io.imsave(f"comparison_image.tif", output_image, imagej=True)

C:\Users\Xianbin\AppData\Local\Temp\ipykernel_21956\1448891147.py:2: UserWarning: comparison_image.tif is a low contrast image
  skimage.io.imsave(f"comparison_image.tif", output_image, imagej=True)
